## Imports

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os
import cv2
from skimage.feature import local_binary_pattern
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from skimage.feature import hog
from sklearn.model_selection import cross_val_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.decomposition import PCA

## Functions

In [12]:
def extract_lbp_features(image):
    num_points = 8
    radius = 1
    gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    # Extrahiere die LBP-Merkmale aus dem Graustufenbild
    lbp = local_binary_pattern(gray_image, num_points, radius, method='ror')
    hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, num_points + 3), range=(0, num_points + 2))
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-7)
    return hist

#def extract_hog_features(image):
 #   gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
 #   hog_features = hog(gray_image, orientations=9, pixels_per_cell=(16, 16), cells_per_block=(2, 2), visualize=False)
 #   return hog_features

def extract_hog_features(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    
    # Erstellen Sie einen HOG-Detektor mit angepassten Parametern
    winSize = (32, 64)  
    blockSize = (8, 8)  
    blockStride = (4, 4)
    cellSize = (4, 4)
    nbins = 9
    hog = cv2.HOGDescriptor(winSize, blockSize, blockStride, cellSize, nbins)
    # Extrahieren Sie die HOG-Merkmale
    hog_features = hog.compute(gray_image, winStride=(8, 8), padding=(8, 8))

    return hog_features

def extract_fisherface_features(image):
    # Convert the image from RGB to Grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    # Load the Fisherface recognizer
    fisherface_recognizer = cv2.FisherFaceRecognizer_create()
    # Extract Fisherface features
    fisherface_feature = fisherface_recognizer.compute(gray_image)
    return fisherface_feature

def predict_image_HOG(image, classifier):
    predict_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # Verkleinere das Bild
    predict_resized_image = cv2.resize(predict_image, (255, 255))
    # Extrahiere die HOG-Merkmale
    hog_features = extract_hog_features(predict_resized_image)
    # Führe die Vorhersage mit dem Klassifikator durch
    prediction = classifier.predict([hog_features])[0]
    # Bestimme die Vertrauenswahrscheinlichkeit für die Vorhersage
    confidence = max(classifier.predict_proba([hog_features])[0])
    return prediction, confidence

def predict_image_LBP(image, classifier):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # Verkleinere das Bild
    resized_image = cv2.resize(image, (255, 255))
    # Extrahiere die LBP-Merkmale
    lbp_features = extract_lbp_features(resized_image)
    # Führe die Vorhersage mit dem Klassifikator durch
    prediction = classifier.predict([lbp_features])[0]
    # Bestimme die Vertrauenswahrscheinlichkeit für die Vorhersage
    confidence = max(classifier.predict_proba([lbp_features])[0])
    return prediction, confidence

def predict_image_Fisherface(image, classifier):
    # Convert the image from BGR to RGB
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # Resize the image to a desired size
    resized_image = cv2.resize(rgb_image, (255, 255))
    # Extract Fisherface features
    fisherface_feature = extract_fisherface_features(resized_image)
    # Perform prediction using the classifier
    prediction = classifier.predict([fisherface_feature])[0]
    # Determine the confidence level for the prediction
    confidence = max(classifier.predict_proba([fisherface_feature])[0])
    return prediction, confidence
    
def detect_face(image_path):
    # Laden des Gesichtserkennungs-Klassifikators (Haar-Cascade)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    # Laden des Bildes
    image = cv2.imread(image_path)
    if image is not None:
        # Konvertieren des Bildes in Graustufen
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        # Gesichter im Bild erkennen
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(200, 200))
        if len(faces) > 0:
            # Nehme das erste erkannte Gesicht
            (x, y, w, h) = faces[0]
            # Schneide das Gesicht aus
            face = image[y:y+h, x:x+w]
             # Zeichne ein Rechteck um das erkannte Gesicht
            ##cv2.rectangle(image, (x, y), (x+w, y+h), (255, 0, 0), 2)
            return face
        else:
            print(f"No face detected in {image_path}")
            return None
    else:
        print(f"Failed to load image: {image_path}")
        return None

# Laden der Bilder und Extrahieren der Merkmale
def load_images_and_extract_features(data_folder, feature_extractor):
    features = []
    labels = []
    for folder_name in os.listdir(data_folder):
        folder_path = os.path.join(data_folder, folder_name)
        if os.path.isdir(folder_path):
            for file_name in os.listdir(folder_path):
                image_path = os.path.join(folder_path, file_name)
                face = detect_face(image_path)
                if face is not None:
                    # Convert the image from BGR to RGB
                    rgb_face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
                    resized_face = cv2.resize(rgb_face, (255, 255))
                    feature = feature_extractor(resized_face)
                    features.append(feature)
                    labels.append(folder_name)
                else:
                    print(f"Failed to detect face in image: {image_path}")
    return np.array(features), np.array(labels)

## Face detection auf zu trainierenden Bildern

In [3]:
#Pfad zum Ordner mit den Unterordnern für jede Person
base_path = "Bilder"

# Zielordner für die geschnittenen Gesichter
output_folder = 'Bilder_detected_faces'

# Erstellen des Zielordners, falls er noch nicht existiert
output_path = output_folder
os.makedirs(output_path, exist_ok=True)

# Durchlaufen der Ordner im Basispfad
for person_folder in os.listdir(base_path):
    person_folder_path = os.path.join(base_path, person_folder)
    # Überprüfen, ob es sich um einen Ordner handelt
    if os.path.isdir(person_folder_path):
        # Durchlaufen der Bilder im Unterordner
        for filename in os.listdir(person_folder_path):
            image_path = os.path.join(person_folder_path, filename)
            # Gesichtserkennung durchführen
            face = detect_face(image_path)
            if face is not None:
                # Pfad für das ausgeschnittene Gesicht
                new_folder_path = os.path.join(output_path, person_folder)
                os.makedirs(new_folder_path, exist_ok=True)
                new_image_path = os.path.join(new_folder_path, f"{filename.split('.')[0]}_face.jpg")
                # Speichern des ausgeschnittenen Gesichts als neues Bild
                cv2.imwrite(new_image_path, face)
                print(f"Gesicht in {filename} wurde erfolgreich ausgeschnitten und als {new_image_path} gespeichert.")
            else:
                print(f"Fehler beim Ausschneiden des Gesichts in {filename}.")

print("Gesichtserkennung und Ausschneiden abgeschlossen.")

Gesicht in 20240403_185035.jpg wurde erfolgreich ausgeschnitten und als Bilder_detected_faces\Nino\20240403_185035_face.jpg gespeichert.
Gesicht in 20240403_185045.jpg wurde erfolgreich ausgeschnitten und als Bilder_detected_faces\Nino\20240403_185045_face.jpg gespeichert.
Gesicht in 20240403_185047.jpg wurde erfolgreich ausgeschnitten und als Bilder_detected_faces\Nino\20240403_185047_face.jpg gespeichert.
Gesicht in 20240403_185056.jpg wurde erfolgreich ausgeschnitten und als Bilder_detected_faces\Nino\20240403_185056_face.jpg gespeichert.
Gesicht in 20240403_185058.jpg wurde erfolgreich ausgeschnitten und als Bilder_detected_faces\Nino\20240403_185058_face.jpg gespeichert.
Gesicht in 20240403_185115.jpg wurde erfolgreich ausgeschnitten und als Bilder_detected_faces\Nino\20240403_185115_face.jpg gespeichert.
Gesicht in 20240403_1851150.jpg wurde erfolgreich ausgeschnitten und als Bilder_detected_faces\Nino\20240403_1851150_face.jpg gespeichert.
Gesicht in 20240403_185146.jpg wurde er

## Erstelle Augmented Bilder

In [11]:
# Define the path to the dataset folder
data_folder = "Bilder_detected_faces"

# Liste zum Speichern der Bilder und ihrer Labels
augmented_images = []
augmented_labels = []

# Iterate over the folders in the dataset folder
for folder_name in os.listdir(data_folder):
    folder_path = os.path.join(data_folder, folder_name)
    # Check if the item is a folder
    if os.path.isdir(folder_path):
        # Iterate over the files in the folder
        for file_name in os.listdir(folder_path):
            # Assuming the files are image files, you can adjust the file type according to your dataset
            # Read the image using OpenCV
            if file_name.lower().endswith(('.jpg', '.jpeg', '.png')):
                image_path = os.path.join(folder_path, file_name)
                image = cv2.imread(image_path)
                # Check if the image is loaded successfully
                if image is not None:
                    # Convert the image from BGR to RGB
                    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    # Resize the image to a desired size
                    resized_image = cv2.resize(rgb_image, (255, 255))
                    # Perform image augmentation using Keras ImageDataGenerator
                    datagen = ImageDataGenerator(
                        rotation_range=20,
                        width_shift_range=0.2,
                        height_shift_range=0.2,
                        shear_range=0.2,
                        zoom_range=0.2,
                        horizontal_flip=True,
                        fill_mode='constant'
                    )
                    # Reshape the image to add batch dimension
                    reshaped_image = resized_image.reshape((1,) + resized_image.shape)
                    # Generate augmented images and append to the list
                    i = 0
                    for batch in datagen.flow(reshaped_image, batch_size=1):
                        augmented_image = batch[0].astype(np.uint8)
                        augmented_images.append(augmented_image)
                        augmented_labels.append(folder_name)  # Hinzufügen des Labels (Ordnername)
                        i += 1
                        if i >= 20:
                            break  # Break the loop after generating 100 augmented images
                else:
                    print(f"Failed to load image: {image_path}")
            else:
                print(f"Skipping non-image file: {file_name}")

# Ausgabe der Anzahl der geladenen Bilder
print(f"Total images: {len(augmented_images)}")
print(f"Total labels: {len(augmented_labels)}")

Total images: 560
Total labels: 560


## Hog Training Feature Extraction

In [13]:
# Initialize empty lists to store HOG features and labels
hog_features = []
hog_labels = []

# Iterate over the images and labels in the lists
for image, label in zip(augmented_images, augmented_labels):
    hog_feature = extract_hog_features(image)
    # Add the HOG feature to the list of features
    hog_features.append(hog_feature)
    # Add the label to the list of labels
    hog_labels.append(label)
    # Print HOG features and label
    print(f"HOG features: {hog_feature}, Label: {label}")

print(f"Total HOG features: {len(hog_features)}")
print(f"Total labels: {len(hog_labels)}")

HOG features: [0.         0.         0.         ... 0.01156801 0.09370038 0.2374027 ], Label: Nino
HOG features: [0.11489123 0.1365105  0.09608502 ... 0.         0.         0.        ], Label: Nino
HOG features: [0. 0. 0. ... 0. 0. 0.], Label: Nino
HOG features: [0.16629298 0.18101498 0.13908851 ... 0.         0.         0.        ], Label: Nino
HOG features: [0.07451055 0.17644547 0.21737874 ... 0.         0.         0.        ], Label: Nino
HOG features: [0. 0. 0. ... 0. 0. 0.], Label: Nino
HOG features: [0.21049581 0.04817683 0.00233348 ... 0.         0.         0.        ], Label: Nino
HOG features: [0.34662008 0.03014974 0.09522872 ... 0.         0.         0.        ], Label: Nino
HOG features: [0.         0.         0.         ... 0.08291367 0.04966503 0.211403  ], Label: Nino
HOG features: [3.3534074e-05 3.0147356e-01 4.2685828e-01 ... 0.0000000e+00 0.0000000e+00
 0.0000000e+00], Label: Nino
HOG features: [0.         0.         0.         ... 0.17333004 0.03483776 0.10505202], 

## LBP Training Feature Extraction

In [14]:
# Initialize empty lists to store LBP features
lbp_features = []
lbp_labels = []

# Parameter für LBP festlegen
num_points = 24
radius = 2

# Iterate over the images and labels in the lists
for image, label in zip(augmented_images, augmented_labels):
    # Extract LBP features
    lbp_feature = extract_lbp_features(image)
    # Flatten the LBP feature array and append to the list of features
    lbp_features.append(lbp_feature)
    # Add the label to the list of labels
    lbp_labels.append(label)
    # Print LBP features and label
    print(f"LBP features: {lbp_feature}, Label: {label}")

print(f"Total LBP features: {len(lbp_features)}")
print(f"Total labels: {len(lbp_labels)}")

LBP features: [0.19789612 0.23991393 0.         0.1913215  0.         0.04482697
 0.         0.31038193 0.         0.01565955], Label: Nino
LBP features: [0.21281741 0.24998409 0.         0.1849424  0.         0.04143066
 0.         0.29262394 0.         0.01820149], Label: Nino
LBP features: [0.19530879 0.24334917 0.         0.18877672 0.         0.04251781
 0.         0.31353919 0.         0.01650831], Label: Nino
LBP features: [0.18847614 0.2240682  0.         0.19668969 0.         0.04025885
 0.         0.33700454 0.         0.01350258], Label: Nino
LBP features: [0.22361395 0.24609401 0.         0.17885161 0.         0.04311425
 0.         0.28887863 0.         0.01944756], Label: Nino
LBP features: [0.20871622 0.24844595 0.         0.18554054 0.         0.04432432
 0.         0.29472973 0.         0.01824324], Label: Nino
LBP features: [0.18186449 0.23824079 0.         0.19052471 0.         0.04058414
 0.         0.33288051 0.         0.01590536], Label: Nino
LBP features: [0.197

## Fisherface Training Feature Extraction

In [15]:
## Fisherface Training Feature Extraction
# Initialize empty lists to store Fisherface features
fisherface_features = []
fisherface_labels = []

# Iterate over the images and labels in the lists
for image, label in zip(augmented_images, augmented_labels):
    # Extract Fisherface features
    fisherface_feature = extract_fisherface_features(image)
    # Append the Fisherface feature to the list of features
    fisherface_features.append(fisherface_feature)
    # Add the label to the list of labels
    fisherface_labels.append(label)
    # Print Fisherface features and label
    print(f"Fisherface features: {fisherface_feature}, Label: {label}")

print(f"Total Fisherface features: {len(fisherface_features)}")
print(f"Total labels: {len(fisherface_labels)}")

AttributeError: module 'cv2' has no attribute 'FisherFaceRecognizer_create'

## Random Forest

In [16]:
## lbp Random Forest
# Laden der Trainingsdaten für LBP
X_train_lbp = lbp_features
y_train_lbp = lbp_labels

# Laden der Testdaten für LBP
X_test_lbp, y_test_lbp = load_images_and_extract_features("Test", lambda img: extract_lbp_features(img))

# Initialisierung des Random-Forest-Klassifikators
rf_classifier_lbp = RandomForestClassifier(n_estimators=300, random_state=42, n_jobs=-1, verbose=0 , max_features='sqrt', criterion='gini')
 
# Durchführung der Kreuzvalidierung mit 5 Faltungen
cv_scores_lbp = cross_val_score(rf_classifier_lbp, X_train_lbp, y_train_lbp, cv=5)

# Trainieren des Random-Forest-Modells auf dem gesamten Trainingsdatensatz
rf_classifier_lbp.fit(X_train_lbp, y_train_lbp)
 
# Evaluierung des Modells auf den Testdaten für LBP
accuracy_lbp = rf_classifier_lbp.score(X_test_lbp, y_test_lbp)
predicted_lbp = rf_classifier_lbp.predict(X_test_lbp)

# Ausgabe der Ergebnisse
print("Cross-Validation Scores lbp:", cv_scores_lbp)
print("Mean Accuracy (Cross-Validation) lbp:", cv_scores_lbp.mean())
print("Accuracy (Test Set) lbp:", accuracy_lbp)
print("Predicted (Test Set) lbp:", predicted_lbp)


## HOG Randon Forest
X_train_hog = hog_features
y_train_hog = hog_labels

# Laden der Testdaten für HOG
X_test_hog, y_test_hog = load_images_and_extract_features("Test", lambda img: extract_hog_features(img))

# Initialisierung des Random-Forest-Klassifikators
rf_classifier_hog = RandomForestClassifier(n_estimators=300, random_state=42, n_jobs=-1, verbose=0, max_features='sqrt', criterion='gini')

# Durchführung der Kreuzvalidierung mit 5 Faltungen
cv_scores = cross_val_score(rf_classifier_hog, X_train_hog, y_train_hog, cv=5)

# Trainieren des Random-Forest-Modells auf dem gesamten Trainingsdatensatz
rf_classifier_hog.fit(X_train_hog, y_train_hog)

# Evaluierung des Modells auf den Testdaten (optional)
accuracy_hog = rf_classifier_hog.score(X_test_hog, y_test_hog)
predicted_hog = rf_classifier_hog.predict(X_test_hog)

# Ausgabe der Ergebnisse
print("Cross-Validation Scores hog:", cv_scores)
print("Mean Accuracy (Cross-Validation) hog:", cv_scores.mean())
print("Accuracy (Test Set) hog:", accuracy_hog)
print("Predicted (Test Set) hog:", predicted_hog)

Cross-Validation Scores lbp: [0.58035714 0.67857143 0.77678571 0.50892857 0.33035714]
Mean Accuracy (Cross-Validation) lbp: 0.5750000000000001
Accuracy (Test Set) lbp: 0.5833333333333334
Predicted (Test Set) lbp: ['Nino' 'Nino' 'Nino' 'Nino' 'Nino' 'Nino' 'Nino' 'Nino' 'Nino' 'Nino'
 'Nino' 'Robin']


ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
2 fits failed with the following error:
joblib.externals.loky.process_executor._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "c:\Users\robin.schlauri\AppData\Local\anaconda3\envs\facerecognition\lib\site-packages\joblib\externals\loky\backend\queues.py", line 125, in _feed
    obj_ = dumps(obj, reducers=reducers)
  File "c:\Users\robin.schlauri\AppData\Local\anaconda3\envs\facerecognition\lib\site-packages\joblib\externals\loky\backend\reduction.py", line 211, in dumps
    dump(obj, buf, reducers=reducers, protocol=protocol)
  File "c:\Users\robin.schlauri\AppData\Local\anaconda3\envs\facerecognition\lib\site-packages\joblib\externals\loky\backend\reduction.py", line 204, in dump
    _LokyPickler

## LDA

In [ ]:
## LDA Klassifizierung

# Laden der Trainingsdaten für LDA
X_train_lda = fisherface_features
y_train_lda = fisherface_labels

# Laden der Testdaten für LDA
X_test_lda, y_test_lda = load_images_and_extract_features("Test", lambda img: extract_fisherface_features(img))

# Initialisierung des LDA-Klassifikators
lda_classifier = LDA()

# Durchführung der Kreuzvalidierung mit 5 Faltungen für LDA
cv_scores_lda = cross_val_score(lda_classifier, X_train_lda, y_train_lda, cv=5)

# Trainieren des LDA-Modells auf dem gesamten Trainingsdatensatz für LDA
lda_classifier.fit(X_train_lda, y_train_lda)

# Evaluierung des Modells auf den Testdaten für LDA
accuracy_lda = lda_classifier.score(X_test_lda, y_test_lda)
predicted_lda = lda_classifier.predict(X_test_lda)

# Ausgabe der Ergebnisse für LDA
print("Cross-Validation Scores lda:", cv_scores_lda)
print("Mean Accuracy (Cross-Validation) lda:", cv_scores_lda.mean())
print("Accuracy (Test Set) lda:", accuracy_lda)
print("Predicted (Test Set) lda:", predicted_lda)

## Bilder predicten

In [ ]:
# Ordnerpfad mit den zu klassifizierenden Bildern
predict_folder = "predict"

# Iteriere über die Bilder im Ordner und führe die Vorhersage durch
for filename in os.listdir(predict_folder):
    image_path = os.path.join(predict_folder, filename)
    # Gesichtserkennung durchführen
    face = detect_face(image_path)
    if face is not None:
        # Plotte das Originalbild mit dem markierten Gesicht
        plt.imshow(cv2.cvtColor(face, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.title('Detected Face')
        plt.show()
        
        # Vorhersage mit LBP durchführen
        prediction, confidence = predict_image_LBP(face, rf_classifier_lbp)
        if prediction is not None:
            print(f"Prediction LBP for {filename}: {prediction}, Confidence: {confidence}")
        
        # Vorhersage mit HOG durchführen
        prediction, confidence = predict_image_HOG(face, rf_classifier_hog)
        if prediction is not None:
            print(f"Prediction HOG for {filename}: {prediction}, Confidence: {confidence}")
        
        # Vorhersage mit Fisherface durchführen
        prediction, confidence = predict_image_Fisherface(face, lda_classifier)
        if prediction is not None:
            print(f"Prediction Fisherface for {filename}: {prediction}, Confidence: {confidence}")
    else:
        print(f"Failed to detect face in {filename}")

## Features Visualisieren

In [ ]:

# Funktion zur Berechnung des Durchschnitts der Merkmale für jede Klasse
def calculate_average_features_by_class(features, labels):
    features_by_class = {}
    class_counts = {}

    # Iteration über Merkmale und Labels
    for feature, label in zip(features, labels):
        if label not in features_by_class:
            features_by_class[label] = [feature]
            class_counts[label] = 1
        else:
            features_by_class[label].append(feature)
            class_counts[label] += 1

    # Berechnung des Durchschnitts der Merkmale für jede Klasse
    average_features = []
    class_labels = []

    for label, features_list in features_by_class.items():
        average_feature = np.mean(features_list, axis=0)
        average_features.append(average_feature)
        class_labels.append(label)

    return np.array(average_features), np.array(class_labels), class_counts

# Berechnung des Durchschnitts der HOG-Merkmale für jede Klasse
average_hog_features_by_class, hog_class_labels, hog_class_counts = calculate_average_features_by_class(hog_features, hog_labels)

# Visualisierung der durchschnittlichen HOG-Merkmale für jede Klasse
plt.figure(figsize=(12, 6))
for i, label in enumerate(hog_class_labels):
    plt.plot(average_hog_features_by_class[i], label=f'Class {label} (n={hog_class_counts[label]})')

plt.xlabel('HOG Feature Index')
plt.ylabel('Average Value')
plt.title('Average HOG Features by Class')
plt.legend()
plt.show()

# Berechnung des Durchschnitts der LBP-Merkmale für jede Klasse
average_lbp_features_by_class, lbp_class_labels, lbp_class_counts = calculate_average_features_by_class(lbp_features, lbp_labels)

# Visualisierung der durchschnittlichen LBP-Merkmale für jede Klasse
plt.figure(figsize=(12, 6))
for i, label in enumerate(lbp_class_labels):
    plt.plot(average_lbp_features_by_class[i], label=f'Class {label} (n={lbp_class_counts[label]})')

plt.xlabel('LBP Feature Index')
plt.ylabel('Average Value')
plt.title('Average LBP Features by Class')
plt.legend()
plt.show()

# Berechnung des Durchschnitts der LDA-Merkmale für jede Klasse
average_lda_features_by_class, lda_class_labels, lda_class_counts = calculate_average_features_by_class(lda_features, labels)

# Visualisierung der durchschnittlichen LDA-Merkmale für jede Klasse
plt.figure(figsize=(12, 6))
for i, label in enumerate(lda_class_labels):
    plt.plot(average_lda_features_by_class[i], label=f'Class {label} (n={lda_class_counts[label]})')

plt.xlabel('LDA Feature Index')
plt.ylabel('Average Value')
plt.title('Average LDA Features by Class')
plt.legend()
plt.show()